In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

client.is_ready()

## Preview data

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.fetch_objects(
    limit=5
)

for item in response.objects:
    print("question:", item.properties["question"])
    print("answer:  ", item.properties["answer"])
    print("points:  ", item.properties["points"])
    print("category:", item.properties["category"])
    print("round:   ", item.properties["round"])
    print("airDate: ", item.properties["airDate"], '\n')

## Vector search

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.near_text(
    query="TV characters",
    limit=5
)

for item in response.objects:
    print(item.properties["question"])
    print(item.properties["answer"])
    print(item.properties["category"], "\n")

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.hybrid(
    query="TV characters",
    limit=5
)

for item in response.objects:
    print(item.properties["question"])
    print(item.properties["answer"])
    print(item.properties["category"], "\n")

## Filters

In [ ]:
from weaviate.classes.query import Filter

wiki = client.collections.get("Questions")

response = wiki.query.fetch_objects(
    filters=Filter.by_property("category").like("TELEVISION"),
    limit=5
)

for item in response.objects:
    print(item.properties)

### Filter by date

* Use `datetime`:
```
datetime(2000, 1, 1).replace(tzinfo=timezone.utc)
```

* or formatted string:
```
"2000-01-01T00:00:00+00:00"
```

In [ ]:
from datetime import datetime, timezone
from weaviate.classes.query import Filter

questions = client.collections.get("Questions")

response = questions.query.fetch_objects(
    limit=5,
    filters= # in the year 2000
        Filter.by_property("airDate").greater_or_equal(datetime(2000, 1, 1).replace(tzinfo=timezone.utc)) &
        Filter.by_property("airDate").less_than(datetime(2001, 1, 1).replace(tzinfo=timezone.utc))
        
    # filters=
    #     Filter.by_property("airDate").greater_or_equal("2000-01-01T00:00:00+00:00") &
    #     Filter.by_property("airDate").less_than("2001-01-01T00:00:00+00:00")
)

for item in response.objects:
    print(item.properties)

## Close the client when done

In [ ]:
client.close()